**Instantiating the MobileNetV2 model**

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(resize_and_rescale)
  model.add(data_augmentation)
  model.add(base_model)
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dense(hp.Choice('filters_1', values=[512, 1024]), activation='relu'))
  model.add(layers.Dense(hp.Choice('filters_2', values=[512, 1024]), activation='relu'))
  # Tune whether to use dropout.
  if hp.Boolean("dropout"):
    model.add(layers.Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))
  model.add(layers.Dense(num_classes, activation='softmax'))
  model.build((None,) + input_shape)
  learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
  # Complitation of the model
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate',
                                                                          values=[1e-2, 1e-3, 1e-4]),
              loss='categorical_crossentropy',
              metrics=['accuracy']))
  return model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import os
from tensorflow.keras import callbacks
from tensorflow.keras.layers import GlobalAveragePooling2D
import io
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import pydot
import graphviz
import seaborn as sns
import shutil
import keras
from matplotlib import pyplot
import keras_tuner

os.chdir("/content")


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Parametri di addestramento
batch_size = 32
input_shape = (224, 224, 3)
image_size = (224, 224)
num_classes = 101
epochs = 15

savingData_dir_name = "./IntelligentSystems/stats"
if not os.path.exists(savingData_dir_name):
  os.makedirs(savingData_dir_name)

# Instantiating the MobileNetV2 pre-trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
# Congelamento dei pesi del modello base
base_model.trainable = False

# Crea un oggetto StringIO per salvare il summary del modello come stringa
summary_string = io.StringIO()

# Salva il summary del modello come stringa
base_model.summary(print_fn=lambda x: summary_string.write(x + '\n'))

# Scrivi la stringa su un file
with open('./IntelligentSystems/stats/model_summary.txt', 'w') as f:
  f.write(summary_string.getvalue())

# Salva una rappresentazione grafica del modello come immagine
tf.keras.utils.plot_model(base_model, to_file='./IntelligentSystems/stats/pre_trained_model_plot.png', show_shapes=True, show_layer_names=True)


**Loading the datasets**

In [ ]:
os.chdir("/content")
# Percorso delle cartelle di train, validation e test
train_dir = "./dataset/food-101/train"
val_dir = "./dataset/food-101/validation"
test_dir = "./dataset/food-101/test"


train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

	# Carica i dati di training
trainGenerator = train_datagen.flow_from_directory(
	train_dir,
	target_size=image_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=True  # Imposta shuffle a True
)

	# Carica i dati di validation
valGenerator = val_datagen.flow_from_directory(
	val_dir,
	target_size=image_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=True  # Imposta shuffle a True
)

	#Carica i dati di test
testGenerator = test_datagen.flow_from_directory(
	test_dir,
	target_size=image_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False  # Mantieni shuffle a False per i dati di test
)

**Making the preprocessing layers**

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(224,224),
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

**Creating the first model**

In [ ]:
first_model = Sequential()
first_model.add(resize_and_rescale)
first_model.add(data_augmentation)
first_model.add(base_model)
first_model.add(layers.GlobalAveragePooling2D())
first_model.add(layers.Dense(512, activation = "relu"))
first_model.add(layers.Dropout(0,2))
first_model.add(layers.Dense(num_classes, activation='softmax'))
first_model.build((None,) + input_shape)
# Complitation of the model
first_model.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
first_model.summary()

# Crea un oggetto StringIO per salvare il summary del modello come stringa
complete_first_model_summary_string = io.StringIO()

# Salva il summary del modello come stringa
first_model.summary(print_fn=lambda x: complete_first_model_summary_string.write(x + '\n'))

# Scrivi la stringa su un file
with open('./IntelligentSystems/stats/complete_first_model_summary.txt', 'w') as f:
  f.write(complete_first_model_summary_string.getvalue())

# Salva una rappresentazione grafica del modello come immagine
tf.keras.utils.plot_model(first_model, to_file='./IntelligentSystems/stats/complete_first_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Ottenere le etichette reali del set di dati di test
labels = testGenerator.classes
# Ottieni l'elenco delle cartelle (classi) nel dataset
class_labels = sorted(os.listdir('./dataset/food-101/test'))

In [ ]:
save_path = os.path.join(savingData_dir_name, 'food_101_pretrained_first_model.h5')
callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath=save_path,
        monitor="val_accuracy",
        verbose=1,
        save_best_only=True,
        mode='max')
]

In [ ]:
history = first_model.fit(
  	trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=valGenerator,
    validation_steps=valGenerator.samples // batch_size)

In [ ]:
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='Training loss', marker='o')
pyplot.plot(history.history['val_loss'], color='orange', label='Validation loss', marker='o', linestyle='--')
pyplot.legend()

# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# Accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history.history['accuracy'], color='blue', label='Training accuracy', marker='o')
pyplot.plot(history.history['val_accuracy'], color='orange', label='Validation accuracy', marker='o', linestyle='--')
pyplot.legend(loc="upper left")

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'first_model_plot.png')
pyplot.show()
pyplot.close()

In [ ]:
scores = first_model.evaluate(testGenerator)
print('Test loss:', scores[0]*100)
print('Test accuracy:', scores[1]*100)

In [ ]:
# Ottenere le predizioni del modello sul set di dati di test
predictions = model.predict(testGenerator, steps = None )

# Calcola la precision
precision = precision_score(labels, predictions.argmax(axis=1), average=None)

# Calcola la recall
recall = recall_score(labels, predictions.argmax(axis=1), average=None)

# Calcolo del punteggio F1
f1Score = f1_score(labels, predictions.argmax(axis=1), average=None)

In [ ]:
f, (ax1, ax2, ax3) = pyplot.subplots(3, sharex=True, figsize=(20, 15))
ax1.bar(class_labels, precision)
ax1.set_title('Precision per classe')
ax2.bar(class_labels, recall)
ax2.set_title('Recall per classe')
ax3.bar(class_labels, f1Score)
ax3.set_title('F1 Score per classe')
pyplot.xticks(rotation=90)
# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'metricsPlotFirstModel.png')
pyplot.show()
pyplot.close()
# Apertura del file in modalità scrittura
with open('./IntelligentSystems/stats/metrics4LabelFirstModel.txt', 'w') as file:
  # Scrittura delle etichette, precisione e recall su file
  for label, precision, recall in zip(class_labels, precision, recall):
    file.write(f'Label: {label}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write('\n')
  file.write(f'F1 Score: {f1Score}\n')

In [ ]:
confusion_mat = confusion_matrix(labels, predictions.argmax(axis=1))
pyplot.figure(figsize=(20, 10))
# Imposta la dimensione del font per le etichette
sns.set(font_scale=1.2)
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Aggiungere etichette agli assi
pyplot.xlabel('Predicted labels')
pyplot.ylabel('True labels')

# Aggiungere un titolo al grafico
pyplot.title('Confusion Matrix')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/stats/' + 'confusionMatrix0.png')
# Mostrare il grafico
pyplot.show()
pyplot.close()

**Tuning the model**

In [ ]:
tuner = keras_tuner.Hyperband(build_model,
                    objective='val_accuracy',
                    max_epochs=20,
                    factor=3,
                    directory='/content/drive/MyDrive/IntelligentSystems',
                    project_name='my_project')

tuner.search(trainGenerator, epochs=50, validation_data=valGenerator, verbose=1 )

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

best_model = tuner.get_best_models(num_models=1)[0]

print(best_hps.get('learning_rate'))
print(best_model.summary())

**Creating the tuned model**

In [ ]:
model = Sequential()
model.add(resize_and_rescale)
model.add(data_augmentation)
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation = "relu"))
model.add(layers.Dense(1024, activation = "relu"))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(num_classes, activation='softmax'))
model.build((None,) + input_shape)
# Complitation of the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

# Crea un oggetto StringIO per salvare il summary del modello come stringa
complete_model_summary_string = io.StringIO()

# Salva il summary del modello come stringa
model.summary(print_fn=lambda x: complete_model_summary_string.write(x + '\n'))

# Scrivi la stringa su un file
with open('./IntelligentSystems/stats/complete_model_summary.txt', 'w') as f:
  f.write(complete_model_summary_string.getvalue())

# Salva una rappresentazione grafica del modello come immagine
tf.keras.utils.plot_model(model, to_file='./IntelligentSystems/stats/complete_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Ottenere le etichette reali del set di dati di test
labels = testGenerator.classes
# Ottieni l'elenco delle cartelle (classi) nel dataset
class_labels = sorted(os.listdir('./dataset/food-101/test'))

**Defining checkpoints**

In [ ]:
save_path = os.path.join(savingData_dir_name, 'food_101_pretrained.h5')
# Definisci il callback EarlyStopping
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath=save_path,
        monitor="val_accuracy",
        verbose=1,
        save_best_only=True,
        mode='max'),
    early_stopping
]

**Training the model**

In [ ]:
# Training the model
history0 = model.fit(
  	trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=valGenerator,
    validation_steps=valGenerator.samples // batch_size)

best_epochs0 = early_stopping.stopped_epoch + 1
print("Best Number of Epochs:", best_epochs0)


**Plotting the results**

In [ ]:
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history0.history['loss'], color='blue', label='Training loss', marker='o')
pyplot.plot(history0.history['val_loss'], color='orange', label='Validation loss', marker='o', linestyle='--')
pyplot.legend()

# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# Accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history0.history['accuracy'], color='blue', label='Training accuracy', marker='o')
pyplot.plot(history0.history['val_accuracy'], color='orange', label='Validation accuracy', marker='o', linestyle='--')
pyplot.legend(loc="upper left")

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'plot.png')
pyplot.show()
pyplot.close()

**Evaluating the new trained model**

In [ ]:
scores0 = model.evaluate(testGenerator)
print('Test loss:', scores0[0]*100)
print('Test accuracy:', scores0[1]*100)

In [ ]:
# Ottenere le predizioni del modello sul set di dati di test
predictions = model.predict(testGenerator, steps = None )

# Calcola la precision
precision = precision_score(labels, predictions.argmax(axis=1), average=None)

# Calcola la recall
recall = recall_score(labels, predictions.argmax(axis=1), average=None)

# Calcolo del punteggio F1
f1Score = f1_score(labels, predictions.argmax(axis=1), average=None)

In [ ]:
f, (ax1, ax2, ax3) = pyplot.subplots(3, sharex=True, figsize=(20, 15))
ax1.bar(class_labels, precision)
ax1.set_title('Precision per classe')
ax2.bar(class_labels, recall)
ax2.set_title('Recall per classe')
ax3.bar(class_labels, f1Score)
ax3.set_title('F1 Score per classe')
pyplot.xticks(rotation=90)
# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'metricsPlot0.png')
pyplot.show()
pyplot.close()
# Apertura del file in modalità scrittura
with open('./IntelligentSystems/stats/metrics4Label0.txt', 'w') as file:
  # Scrittura delle etichette, precisione e recall su file
  for label, precision, recall in zip(class_labels, precision, recall):
    file.write(f'Label: {label}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write('\n')
  file.write(f'F1 Score: {f1Score}\n')

In [ ]:
confusion_mat = confusion_matrix(labels, predictions.argmax(axis=1))
pyplot.figure(figsize=(20, 10))
# Imposta la dimensione del font per le etichette
sns.set(font_scale=1.2)
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Aggiungere etichette agli assi
pyplot.xlabel('Predicted labels')
pyplot.ylabel('True labels')

# Aggiungere un titolo al grafico
pyplot.title('Confusion Matrix')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/stats/' + 'confusionMatrix0.png')
# Mostrare il grafico
pyplot.show()
pyplot.close()

**FINE TUNING - 1**

In [ ]:
base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
    if layer.name == 'Conv_1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()

In [ ]:
save_path2 = os.path.join(savingData_dir_name, 'food_101_pretrained2.h5')

callbacks_list = [
    callbacks.ModelCheckpoint(
    filepath=save_path2,
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode='max'),
    early_stopping
]

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=["accuracy"])

# Training the model
history1 = model.fit(
  	trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=valGenerator,
    validation_steps=valGenerator.samples // batch_size)

best_epochs1 = early_stopping.stopped_epoch + 1
print("Best Number of Epochs:", best_epochs1)

**Plotting the results**

In [ ]:
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history1.history['loss'], color='blue', label='Training loss', marker='o')
pyplot.plot(history1.history['val_loss'], color='orange', label='Validation loss', marker='o', linestyle='--')
pyplot.legend()


# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# Accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history1.history['accuracy'], color='blue', label='Training accuracy', marker='o')
pyplot.plot(history1.history['val_accuracy'], color='orange', label='Validation accuracy', marker='o', linestyle='--')
pyplot.legend(loc="upper left")

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'unfreeze1plot.png')
pyplot.show()
pyplot.close()

In [ ]:
scores1 = model.evaluate(testGenerator)
print('Test loss:', scores1[0]*100)
print('Test accuracy:', scores1[1]*100)

In [ ]:
# Ottenere le predizioni del modello sul set di dati di test
predictions = model.predict(testGenerator, steps = None )

# Calcola la precision
precision = precision_score(labels, predictions.argmax(axis=1), average=None)

# Calcola la recall
recall = recall_score(labels, predictions.argmax(axis=1), average=None)

# Calcolo del punteggio F1
f1Score = f1_score(labels, predictions.argmax(axis=1), average=None)

In [ ]:
f, (ax1, ax2, ax3) = pyplot.subplots(3, sharex=True, figsize=(20, 15))
ax1.bar(class_labels, precision)
ax1.set_title('Precision per classe')
ax2.bar(class_labels, recall)
ax2.set_title('Recall per classe')
ax3.bar(class_labels, f1Score)
ax3.set_title('F1 Score per classe')
pyplot.xticks(rotation=90)
# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'metricsPlot1.png')
pyplot.show()
pyplot.close()
# Apertura del file in modalità scrittura
with open('./IntelligentSystems/stats/metrics4Label1.txt', 'w') as file:
  # Scrittura delle etichette, precisione e recall su file
  for label, precision, recall in zip(class_labels, precision, recall):
    file.write(f'Label: {label}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write('\n')
  file.write(f'F1 Score: {f1Score}\n')

In [ ]:
confusion_mat = confusion_matrix(labels, predictions.argmax(axis=1))
pyplot.figure(figsize=(20, 10))
# Imposta la dimensione del font per le etichette
sns.set(font_scale=1.2)
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Aggiungere etichette agli assi
pyplot.xlabel('Predicted labels')
pyplot.ylabel('True labels')

# Aggiungere un titolo al grafico
pyplot.title('Confusion Matrix')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/stats/' + 'confusionMatrix1.png')
# Mostrare il grafico
pyplot.show()
pyplot.close()

**FINE TUNING - 2**

In [ ]:
base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block_16_project':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()

In [ ]:
save_path3 = os.path.join(savingData_dir_name, 'food_101_pretrained3.h5')

callbacks_list = [
    callbacks.ModelCheckpoint(
    filepath=save_path3,
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode='max'),
    early_stopping
]

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=["accuracy"])
# Training the model
history2 = model.fit(
  	trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=valGenerator,
    validation_steps=valGenerator.samples // batch_size)

best_epochs2 = early_stopping.stopped_epoch + 1
print("Best Number of Epochs:", best_epochs2)

**Plotting the results**

In [ ]:
from matplotlib import pyplot
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history2.history['loss'], color='blue', label='Training loss', marker='o')
pyplot.plot(history2.history['val_loss'], color='orange', label='Validation loss', marker='o', linestyle='--')
pyplot.legend()


# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
# Accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history2.history['accuracy'], color='blue', label='Training accuracy', marker='o')
pyplot.plot(history2.history['val_accuracy'], color='orange', label='Validation accuracy', marker='o', linestyle='--')
pyplot.legend(loc="upper left")

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'unfreeze2plot.png')
pyplot.show()
pyplot.close()

In [ ]:
scores2 = model.evaluate(testGenerator)
print('Test loss:', scores2[0]*100)
print('Test accuracy:', scores2[1]*100)

In [ ]:
# Ottenere le predizioni del modello sul set di dati di test
predictions = model.predict(testGenerator, steps = None )

# Calcola la precision
precision = precision_score(labels, predictions.argmax(axis=1), average=None)

# Calcola la recall
recall = recall_score(labels, predictions.argmax(axis=1), average=None)

# Calcolo del punteggio F1
f1Score = f1_score(labels, predictions.argmax(axis=1), average=None)

In [ ]:
f, (ax1, ax2, ax3) = pyplot.subplots(3, sharex=True, figsize=(20, 15))
ax1.bar(class_labels, precision)
ax1.set_title('Precision per classe')
ax2.bar(class_labels, recall)
ax2.set_title('Recall per classe')
ax3.bar(class_labels, f1Score)
ax3.set_title('F1 Score per classe')
pyplot.xticks(rotation=90)
# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'metricsPlot2.png')
pyplot.show()
pyplot.close()
# Apertura del file in modalità scrittura
with open('./IntelligentSystems/stats/metrics4Label2.txt', 'w') as file:
  # Scrittura delle etichette, precisione e recall su file
  for label, precision, recall in zip(class_labels, precision, recall):
    file.write(f'Label: {label}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write('\n')
  file.write(f'F1 Score: {f1Score}\n')

In [ ]:
confusion_mat = confusion_matrix(labels, predictions.argmax(axis=1))
pyplot.figure(figsize=(20, 10))
# Imposta la dimensione del font per le etichette
sns.set(font_scale=1.2)
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Aggiungere etichette agli assi
pyplot.xlabel('Predicted labels')
pyplot.ylabel('True labels')

# Aggiungere un titolo al grafico
pyplot.title('Confusion Matrix')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/stats/' + 'confusionMatrix2.png')
# Mostrare il grafico
pyplot.show()
pyplot.close()

**FINE TUNING - 3**

In [ ]:
base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block_16_expand':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()

In [ ]:
save_path4 = os.path.join(savingData_dir_name, 'food_101_pretrained4.h5')

callbacks_list = [
    callbacks.ModelCheckpoint(
    filepath=save_path4,
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode='max'),
    early_stopping
]

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=["accuracy"])

# Training the model
history3 = model.fit(
  	trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=valGenerator,
    validation_steps=valGenerator.samples // batch_size)

best_epochs3 = early_stopping.stopped_epoch + 1
print("Best Number of Epochs:", best_epochs3)

**Plotting the results**

In [ ]:
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history3.history['loss'], color='blue', label='Training loss', marker='o')
pyplot.plot(history3.history['val_loss'], color='orange', label='Validation loss', marker='o', linestyle='--')
pyplot.legend()
# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)


# Accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history3.history['accuracy'], color='blue', label='Training accuracy', marker='o')
pyplot.plot(history3.history['val_accuracy'], color='orange', label='Validation accuracy', marker='o', linestyle='--')
pyplot.legend(loc="upper left")

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'unfreeze3plot.png')
pyplot.show()
pyplot.close()

In [ ]:
scores3 = model.evaluate(testGenerator)
print('Test loss:', scores3[0]*100)
print('Test accuracy:', scores3[1]*100)

In [ ]:
# Ottenere le predizioni del modello sul set di dati di test
predictions = model.predict(testGenerator, steps = None )

# Calcola la precision
precision = precision_score(labels, predictions.argmax(axis=1), average=None)

# Calcola la recall
recall = recall_score(labels, predictions.argmax(axis=1), average=None)

# Calcolo del punteggio F1
f1Score = f1_score(labels, predictions.argmax(axis=1), average=None)

In [ ]:
f, (ax1, ax2, ax3) = pyplot.subplots(3, sharex=True, figsize=(20, 15))
ax1.bar(class_labels, precision)
ax1.set_title('Precision per classe')
ax2.bar(class_labels, recall)
ax2.set_title('Recall per classe')
ax3.bar(class_labels, f1Score)
ax3.set_title('F1 Score per classe')
pyplot.xticks(rotation=90)
# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'metricsPlot3.png')
pyplot.show()
pyplot.close()
# Apertura del file in modalità scrittura
with open('./IntelligentSystems/stats/metrics4Label3.txt', 'w') as file:
  # Scrittura delle etichette, precisione e recall su file
  for label, precision, recall in zip(class_labels, precision, recall):
    file.write(f'Label: {label}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write('\n')
  file.write(f'F1 Score: {f1Score}\n')

In [ ]:
confusion_mat = confusion_matrix(labels, predictions.argmax(axis=1))
pyplot.figure(figsize=(20, 10))
# Imposta la dimensione del font per le etichette
sns.set(font_scale=1.2)
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Aggiungere etichette agli assi
pyplot.xlabel('Predicted labels')
pyplot.ylabel('True labels')

# Aggiungere un titolo al grafico
pyplot.title('Confusion Matrix')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/stats/' + 'confusionMatrix3.png')
# Mostrare il grafico
pyplot.show()
pyplot.close()

In [ ]:
pyplot.subplot(211)
pyplot.title('Training Loss Comparison')
pyplot.plot(history0.history['loss'], color='blue', label='Freezed', marker='o')
pyplot.plot(history1.history['loss'], color='orange', label='Tuning-1', marker='o')
pyplot.plot(history2.history['loss'], color='green', label='Tuning-2', marker='o')
pyplot.plot(history3.history['loss'], color='pink', label='Tuning-3', marker='o')
pyplot.legend()

# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# Accuracy
pyplot.subplot(212)
pyplot.title('Training Accuracy Comparison')
pyplot.plot(history0.history['accuracy'], color='blue', label='Freezed', marker='o')
pyplot.plot(history1.history['accuracy'], color='orange', label='Tuning-1', marker='o')
pyplot.plot(history2.history['accuracy'], color='green', label='Tuning-2', marker='o')
pyplot.plot(history3.history['accuracy'], color='pink', label='Tuning-3', marker='o')
pyplot.legend()

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'trainingComparison.png')
pyplot.show()
pyplot.close()

In [ ]:
pyplot.subplot(211)
pyplot.title('Validation Loss Comparison')
pyplot.plot(history0.history['val_loss'], color='blue', label='Freezed', marker='o')
pyplot.plot(history1.history['val_loss'], color='orange', label='Tuning-1', marker='o')
pyplot.plot(history2.history['val_loss'], color='green', label='Tuning-2', marker='o')
pyplot.plot(history3.history['val_loss'], color='pink', label='Tuning-3', marker='o')
pyplot.legend()

# Margine fra i due subplot
pyplot.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# Accuracy
pyplot.subplot(212)
pyplot.title('Validation Accuracy Comparison')
pyplot.plot(history0.history['val_accuracy'], color='blue', label='Freezed', marker='o')
pyplot.plot(history1.history['val_accuracy'], color='orange', label='Tuning-1', marker='o')
pyplot.plot(history2.history['val_accuracy'], color='green', label='Tuning-2', marker='o')
pyplot.plot(history3.history['val_accuracy'], color='pink', label='Tuning-3', marker='o')
pyplot.legend()

if not os.path.exists('./IntelligentSystems/graphs'):
  os.makedirs('./IntelligentSystems/graphs')

# Salva il grafico
pyplot.savefig('./IntelligentSystems/graphs/' + 'validationComparison.png')
pyplot.show()
pyplot.close()